# 1. Lập trình PCA

## 1.1. Tải tập dữ liệu Iris

Import các thư viện cần thiết cho toàn bộ notebook. Đồng thời tải bộ dữ liệu Iris, tách riêng các đặc trưng (features) vào `X` và nhãn (target) vào `y`

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import seaborn as sns
from PCA import *
from Utils.Utils import *
import warnings

warnings.filterwarnings("ignore")

iris = load_iris()
X = pd.DataFrame(iris['data'], columns = iris['feature_names'])
y = pd.DataFrame(iris['target'], columns = ['species'])

## 1.2. Phân tích dữ liệu khám phá (EDA) trên tập Iris

Hiển thị 5 dòng đầu tiên của DataFrame đặc trưng `X` để có cái nhìn tổng quan về dữ liệu.

In [ ]:
X.head()

Cung cấp thông tin tóm tắt về DataFrame `X`, bao gồm kiểu dữ liệu của mỗi cột và số lượng giá trị không null.

In [ ]:
X.info()

Tính toán và hiển thị các thống kê mô tả cho các cột numerical trong DataFrame `X`.

In [ ]:
X.describe()

Kết hợp DataFrame đặc trưng `X` và DataFrame nhãn `y` thành một DataFrame duy nhất `X_all`.

Sau đó vẽ biểu đồ phân tán cặp cho tất cả các đặc trưng, tô màu theo loài hoa (species) và hiển thị histogram trên đường chéo chính để giúp trực quan hóa mối quan hệ giữa các đặc trưng và sự phân bố của chúng theo từng loài.

In [ ]:
X_all = pd.concat([X, y], axis = 1)

sns.pairplot(X_all, hue='species', palette='tab10', diag_kind='hist')

# 2. Áp dụng PCA (MyPCA) trên tập Iris

Chuyển đổi DataFrame thành các mảng NumPy.

In [ ]:
X, y = np.array(X), np.array(y)

Giảm chiều dữ liệu xuống còn 2 thành pnần chính và sau đó khớp mô hình PCA này với `X`. Quá trình `fit` sẽ thực hiện các bước:

<div style="text-align: center;">
    <img src="Images/pca_procedure.png" alt="PCA Procedure" width="600"/>
</div>

In [ ]:
my_pca = MyPCA(n_components = 2)
my_pca.fit(X)

Sử dụng mô hình `MyPCA` đã khớp để biến đổi (transform) dữ liệu đặc trưng `X` ban đầu của tập Iris thành không gian 2 chiều mới. Phép biến đổi được thực hiện bằng cách chiếu dữ liệu đã chuẩn hóa lên các thành phần chính: $X = X_{centered} W$.

In [ ]:
# Transform
X_transformed = my_pca.transform(X)
print("\nDữ liệu sau khi biến đổi (X_transformed) có shape:", X_transformed.shape)
print("Một vài mẫu dữ liệu sau khi biến đổi:")
print(X_transformed[:5, :])

# Kiểm tra kết quả MyPCA với PCA của thư viện Scikit-learn

Thực hiện PCA trên cùng dữ liệu Iris `X` sử dụng triển khai `PCA` từ thư viện `sklearn` với `n_components=2` để so sánh và kiểm chứng kết quả của lớp `MyPCA` tự xây dựng. Đoạn code này khởi tạo, huấn luyện mô hình `SklearnPCA`, biến đổi dữ liệu và in ra các thông số quan trọng như tỷ lệ phương sai giải thích (EVR), phương sai giải thích bởi từng thành phần, và tỷ lệ phương sai giải thích tích lũy (CEVR), cùng với shape và một vài mẫu của dữ liệu đã biến đổi.

In [ ]:
from sklearn.decomposition import PCA as SklearnPCA

print("--- Kiểm tra với sklearn.decomposition.PCA ---")
# 1. Khởi tạo PCA của sklearn
sklearn_pca = SklearnPCA(n_components=2)

# 2. Khớp mô hình với dữ liệu X
sklearn_pca.fit(X)

# 3. Biến đổi dữ liệu X
X_transformed_sklearn = sklearn_pca.transform(X)

print("\nTỷ lệ phương sai giải thích (EVR) cho từng thành phần (sklearn.explained_variance_ratio_):")
# Đây là EVR cho n_components đã chọn
print(sklearn_pca.explained_variance_ratio_)
# Tương đương với self.explained_variance_ratio_[:n_components] của MyPCA

print("\nPhương sai giải thích bởi từng thành phần (sklearn.explained_variance_):")
# Đây là các trị riêng tương ứng với n_components đã chọn, không phải tỷ lệ
print(sklearn_pca.explained_variance_)
# Tương đương với self.explained_variance_[:n_components] của MyPCA

# sklearn không có thuộc tính CEVR trực tiếp, nhưng có thể dễ dàng tính từ EVR
cumulative_explained_variance_ratio_sklearn = np.cumsum(sklearn_pca.explained_variance_ratio_)
print("\nTỷ lệ phương sai giải thích tích lũy (CEVR) tính từ sklearn.explained_variance_ratio_:")
for i, cum_ratio in enumerate(cumulative_explained_variance_ratio_sklearn):
    print(f"  PC1 đến PC{i+1}: {cum_ratio:.4f}")
print(f"Tổng phương sai được giải thích bởi {sklearn_pca.n_components_} thành phần: {cumulative_explained_variance_ratio_sklearn[-1]:.4f}")
# Giá trị cuối cùng này tương đương với self.cumulative_explained_variance_ratio_[n_components-1] của MyPCA

print("\nDữ liệu sau khi biến đổi (X_transformed_sklearn) có shape:", X_transformed_sklearn.shape)
print("Một vài mẫu dữ liệu sau khi biến đổi (sklearn):")
print(X_transformed_sklearn[:5, :]) # Tương đương với X_transformed của MyPCA
print("---------------------------------------------")

# 3. Bài toán Phân cụm K-Means (trên tập ABIDE II)

## 3.1. Tiền xử lý dữ liệu cho tập ABIDE II

Tải bộ dữ liệu ABIDE II từ file CSV (`ABIDE2.csv`). Tách cột 'group' (chứa nhãn Cancer/Normal) làm biến mục tiêu `y`, phần còn lại là đặc trưng `X`.

In [ ]:
X = pd.read_csv('Dataset/ABIDE2.csv', index_col = 0)
y = X['group']

Hiển thị 5 dòng đầu tiên của DataFrame đặc trưng `X` của bộ dữ liệu ABIDE II.

In [ ]:
X.head(5)

Cung cấp thông tin tóm tắt về DataFrame `X` của bộ dữ liệu ABIDE II.

In [ ]:
X.info()

Hiển thị các thống kê mô tả cho các cột số trong DataFrame `X` của bộ dữ liệu ABIDE II.

In [ ]:
X.describe()

Thực thi một quy trình thử nghiệm bao gồm:

* Tiền xử lý dữ liệu (xử lý giá trị thiếu, mã hóa biến hạng mục, loại bỏ các cột có độ lệch thấp, chuẩn hóa)

* Giảm chiều bằng PCA (với 1 thành phần chính), và phân cụm bằng thuật toán K-Means (với `k=2` cụm). 

Mục đích là để đánh giá hiệu suất của K-Means sau khi giảm chiều xuống còn 1 thành phần.

In [ ]:
run_experiment(
    "Dataset/ABIDE2.csv",
    skew_threshold=0.1,
    component_range=[1],
    k=2,
    visualize=False,
    algorithm="KMeans",
    use_poly=False
)

Thực hiện một thử nghiệm tương tự như trên, nhưng lần này giảm chiều dữ liệu xuống còn 2 thành phần chính và sử dụng thêm đặc trưng đa thức trước khi áp dụng K-Means. 

Mục đích là đánh giá xem việc tăng số thành phần PCA và bổ sung đặc trưng đa thức có cải thiện hiệu suất phân cụm không.

In [ ]:
run_experiment(
    "Dataset/ABIDE2.csv",
    skew_threshold=0.1,
    component_range=[2],
    k=2,
    visualize=False,
    algorithm="KMeans",
    use_poly=True
)

Trong thử nghiệm này, thuật toán phân cụm được đổi thành Gaussian Mixture Model (GMM). Dữ liệu được giảm chiều xuống 10 thành phần chính, ngưỡng độ lệch được đặt là 1.0, và không sử dụng đặc trưng đa thức. 

Mục đích là so sánh hiệu suất của GMM với K-Means và đánh giá ảnh hưởng của việc tăng số thành phần PCA cũng như thay đổi ngưỡng xử lý độ lệch.

In [ ]:
run_experiment(
    "Dataset/ABIDE2.csv",
    skew_threshold=1.0,
    component_range=[10],
    k=2,
    visualize=False,
    algorithm="GMM",
    use_poly=False
)

Thử nghiệm này tiếp tục sử dụng thuật toán GMM, nhưng giảm số thành phần chính xuống còn 5, đặt lại ngưỡng độ lệch là 0.1, và lần này có sử dụng đặc trưng đa thức

Mục đích là đánh giá sự kết hợp của GMM, một số lượng thành phần PCA vừa phải, và việc sử dụng đặc trưng đa thức.

In [ ]:
run_experiment(
    "Dataset/ABIDE2.csv",
    skew_threshold=0.1,
    component_range=[5],
    k=2,
    visualize=False,
    algorithm="GMM",
    use_poly=True
)

In [35]:
run_experiment(
    "Dataset/encoded_data.csv",
    component_range=[33],
    algorithm="GMM"
)

=== Experiment started at 20250516_094551 ===
Dataset: Dataset/encoded_data.csv

--- Feature Transformation ---
Loading data from Dataset/encoded_data.csv...
Original data shape: (1004, 129)
Number of numeric features: 128
Number of highly skewed features: 8
Number of features selected for polynomial transformation: 29
Generating interaction terms...
Standardizing features...
Transformed data saved to ./experiment_20250516_094551\transformed_data.csv
Final data shape: (1004, 564)

Transformation Summary:
Visualizations saved to 'feature_transformations.png'
Transformed data shape: (1004, 563)

--- PCA and Clustering Experiments ---

Testing with n_components=33
--- Kết quả Fit ---
Số thành phần chính được chọn: 33
Tỷ lệ phương sai giải thích (EVR):
  PC1: 0.1064
  PC2: 0.0339
  PC3: 0.0262
  PC4: 0.0251
  PC5: 0.0209
  PC6: 0.0191
  PC7: 0.0188
  PC8: 0.0174
  PC9: 0.0168
  PC10: 0.0160
  PC11: 0.0154
  PC12: 0.0149
  PC13: 0.0143
  PC14: 0.0139
  PC15: 0.0131
  PC16: 0.0129
  PC17: 0.

'./experiment_20250516_094551'